## CIFAR10 VGG16 분류기

In [1]:
!conda install -y scikit-learn scikit-image

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3\envs\cvtf2

  added / updated specs:
    - scikit-image
    - scikit-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.0               |       h19a0ad4_0         139 KB
    brotli-1.0.9               |       ha925a31_2         332 KB
    charls-2.1.0               |       h33f27b4_2          92 KB
    cytoolz-0.11.0             |   py37he774522_0         279 KB
    dask-core-2021.2.0         |     pyhd3eb1b0_0         643 KB
    giflib-5.2.1               |       h62dcd97_0          81 KB
    imagecodecs-2021.1.11      |   py37h5da4933_1         5.8 MB
    joblib-1.0.1               |     pyhd3eb1b0_0         208 KB
    lcms2-2.11                 |       hc51a39a_0         451 KB
    lerc-2.2.1                 |       hd77b12b_0         114 KB
    libaec-1.0.4   

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 800

In [5]:
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split

In [6]:
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [9]:
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import Model
from tensorflow.keras.applications import vgg16 as vgg
from tensorflow.keras.layers import Dropout, Flatten, Dense, \
                        GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import np_utils

In [10]:
BATCH_SIZE = 32
NUM_CLASSES = 10
EPOCHS = 25
INPUT_SHAPE = (32, 32, 3)
LEARNING_RATE = 1e-4
MOMENTUM = 0.9

In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [13]:
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train,
                                                 test_size=0.15,
                                                 stratify=np.array(y_train),
                                                 random_state=42)

In [14]:
Y_train = np_utils.to_categorical(y_train, NUM_CLASSES)
Y_val = np_utils.to_categorical(y_val, NUM_CLASSES)
Y_test = np_utils.to_categorical(y_test, NUM_CLASSES)

## 전처리
VGG16 입력 사이즈 48 X 48

In [25]:
from skimage.transform import resize
X_train = np.array([resize(x, (48, 48)) for x in X_train])
X_val = np.array([resize(x, (48, 48)) for x in X_val])
X_test = np.array([resize(x, (48, 48)) for x in x_test])

## 모델 준비
- 최상위층 없이 VGG16 로딩  
- 커스텀 분류기 준비  
- 모델의 맨 위에 새로운 층 쌓기  

In [26]:
base_model = vgg.VGG16(weights = 'imagenet', include_top=False, 
                      input_shape=(48, 48, 3))

In [28]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

분류층만 훈련시키는 것, 훈련할 수 있는 파라미터 세팅을 False로 해서 나머지 층을 동결.  
한습된 가중치를 한 도메인에서 다른 도메인으로 전이 할 수있다.

In [29]:
# VGG16 모델의 세번째 블록에서 마지막 층 추출
last = base_model.get_layer('block3_pool').output

In [31]:
# 상위 층에 분류층 추가
x = GlobalAveragePooling2D()(last)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
pred = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(base_model.input, pred)

In [32]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

## 커스텀 분류기만(전결합층만) 훈련시킬 것이다, VGG16 모델에서 가져온 부분은 동결.

In [33]:
for layer in base_model.layers:
    layer.trainable = False

In [34]:
model.compile(loss='binary_crossentropy',
             optimizer=optimizers.Adam(lr=LEARNING_RATE),
             metrics=['accuracy'])

In [35]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [36]:
# 데이터 늘리기
train_datagen = ImageDataGenerator(rescale=1 / 255.,
                                  horizontal_flip=False)

In [37]:
train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train, Y_train, 
                                    batch_size=BATCH_SIZE)

In [39]:
val_datagen = ImageDataGenerator(rescale=1/255.,
                                 horizontal_flip=False)
val_datagen.fit(X_val)
val_generator = val_datagen.flow(X_val, Y_val,
                                batch_size=BATCH_SIZE)

## 모델 훈련
새로 추가된 층을 훈련시키기 위한 fti_generatir()함수를 호출

In [ ]:
train_steps_per_epoch = X_train.shape[0] // BATCH_SIZE
val_steps_per_epoch = X_val.shape[0] // BATCH_SIZE
history = model.fit_generator(train_generator,
                             steps_per_epoch = train_steps_per_epoch,
                             validation_data=val_generator,
                             validation_steps=val_steps_per_epoch,
                             epochs=EPOCHS,
                             verbose=1)

Epoch 1/25
1328/1328 [==============================] - 137s 102ms/step - loss: 0.3831 - accuracy: 0.2088 - val_loss: 0.2426 - val_accuracy: 0.4611
Epoch 2/25
1328/1328 [==============================] - 138s 104ms/step - loss: 0.2568 - accuracy: 0.4035 - val_loss: 0.2203 - val_accuracy: 0.5087
Epoch 3/25
1328/1328 [==============================] - 138s 104ms/step - loss: 0.2347 - accuracy: 0.4729 - val_loss: 0.2085 - val_accuracy: 0.5406
Epoch 4/25
1328/1328 [==============================] - 138s 104ms/step - loss: 0.2211 - accuracy: 0.5041 - val_loss: 0.2007 - val_accuracy: 0.5610
Epoch 5/25
1328/1328 [==============================] - 138s 104ms/step - loss: 0.2146 - accuracy: 0.5231 - val_loss: 0.1946 - val_accuracy: 0.5740
Epoch 6/25
1328/1328 [==============================] - 138s 104ms/step - loss: 0.2074 - accuracy: 0.5442 - val_loss: 0.1906 - val_accuracy: 0.5807
Epoch 7/25
1328/1328 [==============================] - 139s 105ms/step - loss: 0.2030 - accuracy: 0.5549 - val_

## 모델 성능 분석

In [ ]:
f, (ax1, ax2) = plt.subplot(1, 2, figsize=(15, 5))
t = f.sumtitle('Deep Neural Net Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

In [ ]:
epochs = list(range(1, EPOCHs+1))
ax1.plot(epochs, history.history['accuracy'], label='Train Accuracy')
ax1.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(epochs)
ax1.set_yticks('Accuracy Value')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
l1 = ax1.legend(loc='best')

ax2.plot(epochs, history.history['loss'], label='Train Loss')
ax2.plot(epochs, history.history['val_loss'], label='Validation Loss')
ax2.set_xticks(epochs)
ax2.set_yticks('Loss Value')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
l2 = ax2.legend(loc='best')

In [ ]:
predictions = model.predict(X_test)

In [ ]:
test_labels = list(y_test.squeeze())
predictions = list(predictions.argmax(axis=1))

## 예측 시각화

In [ ]:
def make_prediction(model=None,img_vector=[],
                    label_dict={},top_N=3, 
                    model_input_shape=None):
    if model:
        # get model input shape
        if not model_input_shape:
            model_input_shape = (1,)+model.get_input_shape_at(0)[1:]
            
        # get prediction
        prediction = model.predict(img_vector.reshape(model_input_shape))[0]
        
        
        # get top N with confidence
        labels_predicted = [label_dict[idx] for idx in np.argsort(prediction)[::-1][:top_N]]
        confidence_predicted = np.sort(prediction)[::-1][:top_N]
        
        return labels_predicted, confidence_predicted

def plot_predictions(model,dataset,dataset_labels,label_dict,
                    batch_size,grid_height,grid_width):
    if model:
        f, ax = plt.subplots(grid_width, grid_height)
        f.set_size_inches(12, 12)
        random_batch_indx = np.random.permutation(np.arange(0,len(dataset)))[:batch_size]
        img_idx = 0
        for i in range(0, grid_width):
            for j in range(0, grid_height):
                actual_label = label_dict.get(dataset_labels[random_batch_indx[img_idx]].argmax())
                preds,confs_ = make_prediction(model,
                                              img_vector=dataset[random_batch_indx[img_idx]],
                                              label_dict=label_dict,
                                              top_N=1,
                                              model_input_shape=(1,48,48,3))
                ax[i][j].axis('off')
                ax[i][j].set_title('Actual:'+actual_label[:10]+\
                                    '\nPredicted:'+preds[0] + \
                                    '(' +str(round(confs_[0],2)) + ')')
                ax[i][j].imshow(dataset[random_batch_indx[img_idx]])
                img_idx += 1
        plt.subplots_adjust(left=0, bottom=0, right=1, 
                            top=1, wspace=0.4, hspace=0.55)  

In [ ]:
label_dict = {0:'airplane',
             1:'automobile',
             2:'bird',
             3:'cat',
             4:'deer',
             5:'dog',
             6:'frog',
             7:'horse',
             8:'ship',
             9:'truck'}

In [ ]:
plot_predictions(model=model,dataset=X_test,
                       dataset_labels=Y_test,
                       label_dict=label_dict,
                       batch_size=BATCH_SIZE,
                       grid_height=5,
                       grid_width=5)